# [作業目標]
試著模仿範例寫法, 在鐵達尼生存預測中, 觀察填補缺值以及 標準化 / 最小最大化 對數值的影響


In [117]:
import warnings
warnings.filterwarnings('ignore')

In [118]:
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = 'desktop/data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId','Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'], axis=1)
df = pd.concat([df_train,df_test])
df.head()


,Age,Cabin,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,3,male,1,NaN,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,1,female,1,NaN,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,female,0,NaN,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,1,female,1,NaN,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,3,male,0,NaN,373450


In [119]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

6 Numeric Features : ['Age', 'Fare', 'Parch', 'Pclass', 'SibSp', 'Survived']



In [120]:
# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]
print(train_num)
df.head()

891


,Age,Fare,Parch,Pclass,SibSp,Survived
0,22.0,7.2500,0,3,1,NaN
1,38.0,71.2833,0,1,1,NaN
2,26.0,7.9250,0,3,0,NaN
3,35.0,53.1000,0,1,1,NaN
4,35.0,8.0500,0,3,0,NaN


作業1
試著在補空值區塊, 替換並執行兩種以上填補的缺值, 看看何者比較好?

In [121]:
# 空值補 -1, 做羅吉斯迴歸
df_m1 = df.fillna(-1)
train_X = df_m1[:train_num]
estimator = LogisticRegression(solver='lbfgs')

cross_val_score(estimator, train_X, train_Y, cv=5).mean()


0.6982644788418415

In [122]:
# 空值補 0
df_0 = df.fillna(0)
train_X = df_0[:train_num]
estimator = LogisticRegression(solver='lbfgs')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6993817972775958

In [123]:
# 空值補平均值
df_mn = df.fillna(df.mean())
train_X = df_mn[:train_num]
estimator = LogisticRegression(solver='lbfgs')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6959413955734954

作業2
使用不同的標準化方式 (原值 / 最小最大化 / 標準化)，搭配羅吉斯迴歸模型，何者效果最好?
Ans:因LogisticRegression為依據樣本分佈機率進行運作,因此標準化條件下(標準差不變的情況下),模型解釋解力都為一致.

In [129]:
# 原值
df = df.notnull()
#print(df)
#print(df.isnull().sum())
train_X = df[:train_num]
estimator = LogisticRegression(solver='lbfgs')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()


Age         0
Fare        0
Parch       0
Pclass      0
SibSp       0
Survived    0
dtype: int64


0.6161649089097865

In [125]:
# 搭配標準化
df = df[~df.isnull()]
df_temp = StandardScaler().fit_transform(df)
train_X = df_temp[:train_num]
estimator = LogisticRegression(solver='lbfgs')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6161649089097865

In [126]:
#  搭配最大最小化
df_temp = MinMaxScaler().fit_transform(df)
train_X = df_temp[:train_num]
estimator = LogisticRegression(solver='lbfgs')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6161649089097865